In [ ]:
apifrom fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
#langserve is a LangChain extension library. It helps you expose your LangChain chains as REST API endpoints automatically.
#add_routes is a helper function inside langserve.It registers your LangChain chain (or multiple chains) as API endpoints on your FastAPI app.
from langserve import add_routes
#uvicorn is a lightweight ASGI web server for running FastAPI (or any ASGI-based app).
#Without uvicorn, your FastAPI app is just Python code — uvicorn actually starts the web server to listen for incoming HTTP requests.
import uvicorn
import os
from langchain_community.llms import Ollama
from dotenv import load_dotenv

In [ ]:
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

#You are initializing a FastAPI application object, stored in variable app. This object will hold your entire API server.
app=FastAPI(
    title="Langchain Server", #The name of your API (will show in docs)
    version="1.0",         #Version number of your API
    decsription="A simple API Server"    #Description shown in auto-generated documentation (Swagger UI)

)


In [ ]:
add_routes(   #From langserve — automatically exposes a LangChain chain as a REST API route in FastAPI.
    app, # 	Your FastAPI app object you created earlier (app = FastAPI())
    ChatOpenAI(),  #You are initializing a LangChain OpenAI model (default uses gpt-3.5-turbo unless you specify).
    path="/openai"  #The API endpoint path. This means you can send requests to http://localhost:8000/openai/invoke.
)
model=ChatOpenAI()
##ollama llama2
llm=Ollama(model="llama2")


Tool	Role

uvicorn-	Runs your server (ASGI server)

FastAPI-	Builds your API + Swagger UI

langserve-	Exposes LangChain chains as routes

In [ ]:

prompt1=ChatPromptTemplate.from_template("Write me an essay about {topic} with 100 words")
prompt2=ChatPromptTemplate.from_template("Write me an poem about {topic} for a 5 years child with 100 words")

add_routes(
    app,
    prompt1|model,   # OpenAI ChatOpenAI() model.
    path="/essay"


)

add_routes(
    app,
    prompt2|llm,   # Ollama(model="llama2") model.
    path="/poem"


)


In [ ]:
if __name__=="__main__":
    uvicorn.run(app,host="localhost",port=8000)


#creating streamlit app for response


In [ ]:
import requests
import streamlit as st

def get_openai_response(input_text):
    response=requests.post("http://localhost:8000/essay/invoke",   #Sends data to your LangChain API
    json={'input':{'topic':input_text}})   #The input for your prompt

    return response.json()['output']['content']     #Extracts the generated essay

    def get_ollama_response(input_text):
    response=requests.post(
    "http://localhost:8000/poem/invoke",
    json={'input':{'topic':input_text}})

    return response.json()['output']

    ## streamlit framework

st.title('Langchain Demo With LLAMA2 API')
input_text=st.text_input("Write an essay on")
input_text1=st.text_input("Write a poem on")

if input_text:
    st.write(get_openai_response(input_text))

if input_text1:
    st.write(get_ollama_response(input_text1))
